<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/SQL/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B8_%D0%BF%D0%BE_SQL_%D1%81_%D0%BE%D0%BA%D0%BE%D0%BD%D0%BD%D1%8B%D0%BC%D0%B8_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Разбор SQL задач с собеседований
## Тема: Оконные функции

## Кратко про оконные функции в SQL

**Оконные функции** — это инструмент SQL, позволяющий выполнять сложные вычисления **без потери исходных данных**. В отличие от агрегатных функций (`SUM`, `AVG`), которые группируют строки в одну, оконные функции добавляют результаты к каждой строке, сохраняя её уникальность. Это особенно полезно для анализа временных рядов, расчёта скользящих средних, ранжирования и вычисления долей.

---

### **Как работают оконные функции**
1. **Разделение на партиции** (`PARTITION BY`)  
   Данные делятся на группы по заданным критериям (например, по категориям или регионам). Для каждой группы выполняется отдельный расчёт.

2. **Упорядочивание** (`ORDER BY`)  
   Строки внутри партиции сортируются. Например, по дате для расчёта кумулятивных сумм.

3. **Определение окна** (`ROWS` или `RANGE`)  
   Указывается диапазон строк, участвующих в вычислении. Например, `ROWS BETWEEN 6 PRECEDING AND CURRENT ROW` — 7 дней.

---

### **Ключевые преимущества**
- **Эффективность**: Замена подзапросов и соединений.  
- **Читаемость**: Короткие запросы с явным синтаксисом.  
- **Гибкость**: Комбинация с агрегатными функциями (`SUM`, `AVG`) и аналитическими (`LAG`, `LEAD`).

---

### **Типы оконных функций**
1. **Агрегатные** (`SUM`, `AVG`, `MAX`)  
   Повторяют стандартные агрегатные функции, но сохраняют строки.  
2. **Ранжирующие** (`RANK`, `DENSE_RANK`)  
   Присваивают ранги строкам.  
3. **Функции смещения** (`LAG`, `LEAD`)  
   Доступ к соседним строкам.  


Переходим к разбору задач с собеседований на тему Оконные функции

## Дана таблица **sales**  
**Поля:**  
1. **product_id** — уникальный идентификатор товара.  
2. **category_id** — категория товара (например, «Электроника», «Одежда»).  
3. **sale_date** — дата продажи.  
4. **sales_amount** — сумма продажи.  
5. **region** — регион продаж (например, «Север», «Юг»).  

**Пример данных:**  
```sql
+------------+-------------+------------+--------------+--------+
| product_id | category_id | sale_date  | sales_amount | region |
+------------+-------------+------------+--------------+--------+
| 1          | Electronics | 2025-03-01 | 1500.00      | North  |
| 2          | Electronics | 2025-03-02 | 2000.00      | North  |
| 3          | Clothing    | 2025-03-01 | 800.00       | South  |
+------------+-------------+------------+--------------+--------+
```

### **Кейс 1: Анализ продаж по категориям**  
**Задача:**  
*«Найдите топ-3 продуктов в каждой категории по продажам. Если два продукта имеют одинаковую сумму продаж, они должны получить одинаковый ранг»*.  

**Решение:**  
```sql
SELECT
    product_id,
    category_id,
    sales_amount,
    RANK() OVER (PARTITION BY category_id ORDER BY sales_amount DESC) AS sales_rank
FROM sales
WHERE sales_rank <= 3;
```
---

### **Пошаговый разбор**

#### **1. Окно RANK() OVER()**
```sql
RANK() OVER (
    PARTITION BY category_id
    ORDER BY sales_amount DESC
)
```
- **PARTITION BY category_id**: Группировка по категориям.
- **ORDER BY sales_amount DESC**: Сортировка внутри каждой категории по сумме продаж в **убывающем порядке**.
- **RANK()**: Назначает ранг, учитывая **одинаковые значения** (например, два продукта с одинаковой суммой продаж получат одинаковый ранг, а следующий продукт будет рангом на 1 выше).

#### **2. Фильтрация результатов**
```sql
WHERE sales_rank <= 3
```
- Выбирает только продукты с рангом **1, 2 или 3** в своей категории.

---

### **Пример работы**
Допустим, в категории **A** есть продукты с продажами:
```sql
| product_id | sales_amount |
|------------|--------------|
| 1          | 1000         |
| 2          | 800          |
| 3          | 800          |
| 4          | 700          |
```

**Результат**:
```sql
| product_id | category_id | sales_amount | sales_rank |
|------------|-------------|--------------|-----------|
| 1          | A           | 1000         | 1         |
| 2          | A           | 800          | 2         |
| 3          | A           | 800          | 2         |
```

**Почему?**
- Продукты **2** и **3** получают ранг **2**, так как их продажи равны.
- Следующий продукт (ранг **3**) отсутствует, поэтому в топ-3 попадают только три продукта.

---

### **Особенности решения**
1. **Равные ранги**: `RANK()` автоматически обрабатывает одинаковые значения, не пропуская ранги (в отличие от `ROW_NUMBER()`).
2. **Группировка**: `PARTITION BY` гарантирует, что топ-3 считается **отдельно для каждой категории**.
3. **Фильтрация**: `WHERE sales_rank <= 3` отсекает продукты вне топ-3.

---

### **Что делать, если нужно включить все продукты при равных рангах?**
Если в категории есть продукты с одинаковым рангом **3**, но их больше трёх, используйте `DENSE_RANK()` или `RANK()` с дополнительной фильтрацией. Например:
```sql
SELECT * FROM (
    SELECT
        product_id,
        category_id,
        sales_amount,
        RANK() OVER (PARTITION BY category_id ORDER BY sales_amount DESC) AS sales_rank
    FROM sales
) AS subquery
WHERE sales_rank <= 3;
```

---


### **Кейс 2: Кумулятивные продажи по регионам**  
**Задача:**  
*«Рассчитайте ежедневный объем продаж для каждого региона, начиная с первого дня продаж»*.  

**Решение:**  
```sql
SELECT
    region,
    sale_date,
    sales_amount,
    SUM(sales_amount) OVER (PARTITION BY region ORDER BY sale_date) AS cumulative_sales
FROM sales;
```
---

### **Пошаговое объяснение**
#### **1. `SELECT` — выборка данных**
- **`region`, `sale_date`, `sales_amount`**  
  Выбираем исходные столбцы для отображения ежедневных продаж.

#### **2. `SUM(sales_amount) OVER (...)` — кумулятивная сумма**
- **`SUM(sales_amount)`**  
  Суммирует значения `sales_amount`.
- **`OVER (PARTITION BY region ORDER BY sale_date)`**  
  **`PARTITION BY region`**:  
  Разделяет данные на группы по регионам. Для каждого региона будет отдельный подсчёт.  
  **`ORDER BY sale_date`**:  
  Устанавливает порядок расчёта внутри каждой группы. Сумма накапливается от старейшей даты к новейшей.

---

### **Пример работы запроса**
**Данные в таблице `sales`:**
```sql
| region | sale_date   | sales_amount |
|--------|-------------|--------------|
| Север  | 2023-01-01  | 1000         |
| Север  | 2023-01-02  | 1200         |
| Север  | 2023-01-03  | 1500         |
| Юг     | 2023-01-01  | 800          |
| Юг     | 2023-01-02  | 900          |
| Юг     | 2023-01-03  | 1100         |
```

**Результат запроса:**
```sql
| region | sale_date   | sales_amount | cumulative_sales |
|--------|-------------|--------------|------------------|
| Север  | 2023-01-01  | 1000         | 1000         |
| Север  | 2023-01-02  | 1200         | 2200         |
| Север  | 2023-01-03  | 1500         | 3700         |
| Юг     | 2023-01-01  | 800          | 800          |
| Юг     | 2023-01-02  | 900          | 1700         |
| Юг     | 2023-01-03  | 1100         | 2800         |
```
---

### **Особенности и нюансы**
#### **1. Ежедневный объём vs кумулятивный**
- **`sales_amount`** — показывает продажи за конкретный день.  
- **`cumulative_sales`** — сумма продаж с первого дня до текущего.

#### **2. Дубликаты в данных**
Если в таблице есть **дубликаты** (например, несколько записей за один день), добавьте группировку:
```sql
SELECT
    region,
    sale_date,
    SUM(sales_amount) AS daily_sales,  -- Суммируем дубликаты
    SUM(SUM(sales_amount)) OVER (PARTITION BY region ORDER BY sale_date) AS cumulative_sales
FROM sales
GROUP BY region, sale_date;
```

---

### **Визуализация работы запроса**
**Для региона «Север»:**
1. **2023-01-01**:  
   `cumulative_sales = 1000` (первая дата → сумма равна `sales_amount`).  
2. **2023-01-02**:  
   `cumulative_sales = 1000 + 1200 = 2200`.  
3. **2023-01-03**:  
   `cumulative_sales = 2200 + 1500 = 3700`.

---

### **Что делать, если...**
#### **1. Нужен только ежедневный объём**
Уберите `cumulative_sales` из запроса:
```sql
SELECT
    region,
    sale_date,
    sales_amount
FROM sales;
```

#### **2. Даты не отсортированы**
Добавьте `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW` для явного указания диапазона:
```sql
SUM(sales_amount) OVER (
    PARTITION BY region
    ORDER BY sale_date
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
) AS cumulative_sales
```

---

### **Кейс 3: Скользящее среднее за 7 дней**  
**Задача:**  
*«Найдите среднюю сумму продаж для каждого продукта за последние 7 дней»*.  

**Решение:**  
```sql
SELECT
    product_id,
    sale_date,
    sales_amount,
    AVG(sales_amount) OVER (
        PARTITION BY product_id
        ORDER BY sale_date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS rolling_avg
FROM sales;
```
---

### **Пошаговое объяснение**
#### **1. `PARTITION BY product_id`**
Группирует данные по продуктам. Для каждого продукта будет отдельный расчёт скользящего среднего.

#### **2. `ORDER BY sale_date`**
Устанавливает порядок расчёта: среднее считается **от текущей даты и 6 предыдущих дней**.

#### **3. `ROWS BETWEEN 6 PRECEDING AND CURRENT ROW`**
Определяет окно расчёта:
- **`6 PRECEDING`**: берёт 6 дней **до текущей даты**.
- **`CURRENT ROW`**: включает текущий день.
- **Итого**: 7 дней (6 предыдущих + текущий).

---

### **Пример работы запроса**
**Данные в таблице `sales`:**
```sql
| product_id | sale_date   | sales_amount |
|------------|-------------|--------------|
| 1          | 2023-01-01  | 100          |
| 1          | 2023-01-02  | 120          |
| 1          | 2023-01-03  | 150          |
| 1          | 2023-01-04  | 180          |
| 1          | 2023-01-05  | 200          |
| 1          | 2023-01-06  | 220          |
| 1          | 2023-01-07  | 250          |
| 1          | 2023-01-08  | 280          |
```
**Результат запроса:**
```sql
| product_id | sale_date   | sales_amount | rolling_avg |
|------------|-------------|--------------|-------------|
| 1          | 2023-01-01  | 100          | 100         |
| 1          | 2023-01-02  | 120          | 110         |
| 1          | 2023-01-03  | 150          | 135         |
| 1          | 2023-01-04  | 180          | 165         |
| 1          | 2023-01-05  | 200          | 195         |
| 1          | 2023-01-06  | 220          | 225         |
| 1          | 2023-01-07  | 250          | 255         |
| 1          | 2023-01-08  | 280          | 285         |
```
---

### **Особенности и нюансы**
#### **1. Первые 6 дней**
Для дней **до 7-го** скользящее среднее будет рассчитываться на меньшем количестве дней:
- **День 1**: среднее = 100 (1 день).  
- **День 2**: среднее = (100 + 120)/2 = 110 (2 дня).  
- **День 3**: среднее = (100 + 120 + 150)/3 = 123.33 (3 дня).  
- **...**  
- **День 7**: среднее = (100 + 120 + ... + 250)/7 = 255 (7 дней).

#### **2. Дубликаты в данных**
Если в таблице есть **дубликаты** (например, несколько записей за один день), добавьте группировку:
```sql
SELECT
    product_id,
    sale_date,
    SUM(sales_amount) AS daily_sales,
    AVG(SUM(sales_amount)) OVER (
        PARTITION BY product_id
        ORDER BY sale_date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS rolling_avg
FROM sales
GROUP BY product_id, sale_date;
```

---

### **Визуализация работы запроса**
**Для продукта с `product_id = 1`:**
1. **2023-01-01**:  
   `rolling_avg = 100` (только текущий день).  
2. **2023-01-02**:  
   `rolling_avg = (100 + 120)/2 = 110`.  
3. **2023-01-03**:  
   `rolling_avg = (100 + 120 + 150)/3 ≈ 123.33`.  
4. **...**  
7. **2023-01-07**:  
   `rolling_avg = (100 + 120 + ... + 250)/7 = 255`.  
8. **2023-01-08**:  
   `rolling_avg = (120 + 150 + ... + 280)/7 ≈ 285`.

---

### **Что делать, если...**
#### **1. Нужно изменить период**
Замените `6 PRECEDING` на нужное количество дней:
```sql
ROWS BETWEEN 14 PRECEDING AND CURRENT ROW  -- 15-дневное окно
```

#### **2. Даты не отсортированы**
Добавьте явное указание диапазона:
```sql
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
```


### **Кейс 4: Доля продукта в категории**  
**Задача:**  
*«Рассчитайте, какую долю от общих продаж в категории составляет каждый продукт»*.  

**Решение:**  
```sql
SELECT
    product_id,
    category_id,
    sales_amount,
    sales_amount * 100.0 / SUM(sales_amount) OVER (PARTITION BY category_id) AS sales_percentage
FROM sales;
```
---


### **Пошаговое объяснение**
#### **1. `PARTITION BY category_id`**
Группирует данные по категориям. Для каждой категории будет отдельный расчёт общих продаж.

#### **2. `SUM(sales_amount) OVER (...)`**
Суммирует все продажи **в категории**.  
Пример:  
Если в категории `category_id = 1` есть продукты с продажами 100, 200, 300, то `SUM(sales_amount) OVER (...)` вернёт **600**.

#### **3. `sales_amount * 100.0 / ...`**
Вычисляет долю продаж продукта в категории.  
**`100.0`** используется для точного деления (избегает округления до целого числа).

---

### **Пример работы запроса**
**Данные в таблице `sales`:**
```sql
| product_id | category_id | sales_amount |
|------------|-------------|--------------|
| 1          | 1           | 100          |
| 2          | 1           | 200          |
| 3          | 1           | 300          |
| 4          | 2           | 500          |
| 5          | 2           | 600          |
```
**Результат запроса:**
```sql
| product_id | category_id | sales_amount | sales_percentage |
|------------|-------------|--------------|------------------|
| 1          | 1           | 100          |   16.67          |
| 2          | 1           | 200          |   33.33          |
| 3          | 1           | 300          |   50.00          |
| 4          | 2           | 500          |   45.45          |
| 5          | 2           | 600          |   54.55          |
```
---

### **Особенности и нюансы**
#### **1. Округление результатов**
Для округления до целого числа используйте `ROUND()`:
```sql
ROUND(sales_amount * 100.0 / SUM(sales_amount) OVER (PARTITION BY category_id), 2) AS sales_percentage
```

#### **2. Дубликаты в данных**
Если в таблице есть **дубликаты** (например, несколько записей для одного продукта в категории), добавьте группировку:
```sql
SELECT
    product_id,
    category_id,
    SUM(sales_amount) AS total_sales,
    SUM(sales_amount) * 100.0 / SUM(SUM(sales_amount)) OVER (PARTITION BY category_id) AS sales_percentage
FROM sales
GROUP BY product_id, category_id;
```

---

### **Визуализация работы запроса**
**Для категории `category_id = 1`:**
1. **Общие продажи категории**:  
   `SUM(sales_amount) = 100 + 200 + 300 = 600`.  
2. **Доля продукта `product_id = 1`**:  
   `(100 / 600) × 100 = 16.67%`.  
3. **Доля продукта `product_id = 2`**:  
   `(200 / 600) × 100 = 33.33%`.  
4. **Доля продукта `product_id = 3`**:  
   `(300 / 600) × 100 = 50.00%`.

---

### **Что делать, если...**
#### **1. Нужно отсортировать результаты**
Добавьте сортировку по доле продаж:
```sql
SELECT
    product_id,
    category_id,
    sales_amount,
    sales_amount * 100.0 / SUM(sales_amount) OVER (PARTITION BY category_id) AS sales_percentage
FROM sales
ORDER BY category_id, sales_percentage DESC;
```

#### **2. Даты не учитываются**
Если продажи разбросаны по дням, используйте группировку по дате:
```sql
SELECT
    product_id,
    category_id,
    sale_date,
    sales_amount,
    sales_amount * 100.0 / SUM(sales_amount) OVER (PARTITION BY category_id, sale_date) AS daily_sales_percentage
FROM sales;
```

---

## Заключение

**Оконные функции** — это **ключевой инструмент** для решения задач анализа данных, где требуется сохранить исходные строки при выполнении сложных вычислений. Они позволяют:  
1. **Рассчитывать кумулятивные суммы** (например, ежедневный объём продаж за период).  
2. **Вычислять скользящие средние** (например, за 7 дней).  
3. **Определять доли** (например, долю продукта в категории).  
4. **Ранжировать данные** (например, топ-продукты по продажам).  

**Зачем это нужно аналитику?**  
- **Эффективность**: Замена подзапросов и соединений сокращает время выполнения запросов.  
- **Читаемость**: Явный синтаксис упрощает понимание логики расчётов.  
- **Гибкость**: Комбинация с другими функциями (`LAG`, `LEAD`, `RANK`) открывает путь к решению задач, связанных с временными рядами, прогнозированием и визуализацией данных.  

**Итог**: Освоение оконных функций — **обязательный навык** для аналитика, работающего с SQL. Это позволяет быстро и точно отвечать на бизнес-вопросы, связанные с динамикой данных, ранжированием и долевым анализом.

---